In [ ]:
import matplotlib
matplotlib.use("PDF")
import pyfits as pf
from pylab import *
import sys, glob
from scipy import stats

thedir='/sandbox/lsst/lsst/GUI/20161011_029-cte-voltages
%cd $thedir

#%matplotlib inline

In [ ]:
files = sort(glob.glob('ITL-3800C-029_flat_light_0??_20161011??????.fits'))
name = files[0][25:28]
print files
print name

In [ ]:
xstart = 505
xstop = 542
ov_start = 512
ystart = 200
ystop = 1800
xaxis = linspace(xstart,xstop-1,xstop-xstart)
outfile = open('cte.txt', 'w')
for file in files:
    name = file[25:28]
    hdulist = pf.open(file, mode='readonly', do_not_scale_image_data=True)
    v_hdr = hdulist[18].header
    v_keys = ['V_S1L', 'V_S1H', 'V_RGL', 'V_RGH', 'V_OG1', 'V_RD1']
    v_string = ""
    for key in v_keys:
        v_string += "%s = %s;"%(key,v_hdr[key])
    if int(name) % 2 == 0:
        outfile.write(v_string+' High Flux\n')
    else:
        outfile.write(v_string+' Low Flux\n')
    figure(figsize=(15,15))
    subplots_adjust(hspace = 0.5, wspace = 0.5)
    suptitle("Serial CTE by segment - ITL CCD 029\n%s"%v_string, fontsize = 16)

    for segment in range(16):#[8]:
        try:
            subplot(4,4,segment+1)
            imhdr=hdulist[segment+1].header
            extname = imhdr['EXTNAME']
            title("CTE - %s"%extname)
            data = array(hdulist[segment+1].data + 32768, dtype = int32)
            flat_overscan = array(data[:,xstop-8:xstop].sum(axis = 1) / 8.0)
            cte_data = ((transpose(transpose(data) - flat_overscan))[ystart:ystop,:].mean(axis=0))[xstart:xstop]
            cte_std = ((transpose(transpose(data) - flat_overscan))[ystart:ystop,:].std(axis=0) / sqrt(float(ystop-ystart)))[xstart:xstop]
            cti = (transpose(transpose(data) - flat_overscan))[ystart:ystop,ov_start].mean()\
            / (transpose(transpose(data) - flat_overscan))[ystart:ystop,ov_start-1].mean() / ov_start
            scatter(xaxis, cte_data, color = 'blue')
            errorbar(xaxis, cte_data, yerr=cte_std)
            text(xstart + 8, 20000, "CTI = %.2E"%cti, fontsize = 12)
            yscale('log',nonposy='clip')
            ylim(.01, 100000)
            xticks([510,520,530,540])
            xlabel("X (First overscan = 512)")
            ylabel("Signal(ADU)")
            outfile.write('%s, CTI = %.3E\n'%(extname,cti))
            print "Segment %s done"%extname
        except:
            continue
    savefig("Serial_CTE_%s_11Oct16.pdf"%name)
    clf()
outfile.close()

In [ ]:
hdulist = pf.open(files[0], mode='readonly', do_not_scale_image_data=True)
infile = open('cte.txt', 'r')
lines = infile.readlines()
infile.close()
ctis = zeros([16,13,2])
flux = 1
voltage = -1
for line in lines:
    if list(line)[0] == 'V':
        if flux == 1:
            flux = 0
            voltage += 1
        else:
            flux = 1
        continue
    seg = line.split()[0].strip(',')
    for segment in range(16):
        imhdr=hdulist[segment+1].header
        extname = imhdr['EXTNAME']
        if extname == seg:
            break
    cti = float(line.split()[3])
    ctis[segment,voltage,flux] = cti
fluxes = ['Low_Flux', 'High_Flux']
xaxis = linspace(1,13,13)
for flux_index,flux in enumerate(fluxes):
    figure(figsize=(15,15))
    subplots_adjust(hspace = 0.5, wspace = 0.5)
    suptitle("Serial CTE by segment - ITL CCD 029 - %s"%flux, fontsize = 16)
    for segment in range(16):
        subplot(4,4,segment+1)
        imhdr=hdulist[segment+1].header
        extname = imhdr['EXTNAME']
        title("CTE - %s"%extname)
        scatter(xaxis,ctis[segment,:,flux_index])
        yscale('log',nonposy='clip')
        ylim(1.0E-7, 1.0E-2)
        xlabel("Voltage Combinations")
        ylabel("CTI")
    savefig("Serial_CTE_Summary_%s_11Oct16.pdf"%flux)
    clf()